# Exam: Scraping job postings from Jobindex

....

In [3]:
import scraping_class
logfile = 'log_jobindex_scraping.txt'
connector = scraping_class.Connector(logfile)

In [4]:
import requests
from bs4 import BeautifulSoup
import numpy as np

#define url and fetch the HTML using the requests module
url = 'https://www.jobindex.dk/jobsoegning?lang=da.html'
response = requests.get(url)  
html = response.text  
soup = BeautifulSoup(html,'lxml') # parse the raw html using BeautifoulSoup
#print(soup)

In [6]:
#find the max page number
max_page = soup.find('ul',{'class':'pagination'})
max_page2 = max_page.find_all('a', {'class':'page-link'})

def convert_value_type(value_node):
    if value_node.name == 'a':
        return value_node.text

page_list = []
for page in max_page2:
    page_list.append(convert_value_type(page))
#print(page_list[-1])

last_page = int(page_list[-1]) + 1 # we add one to use it in range in for-loop
print(last_page)

648


In [22]:
#check exercise 6.1.4??
import re
#Get all link pages from jobindex
jobindex_links = []
for i in range(1,3): # NOTE I'm only testing it on the first two pages. Use last_page when ready
    url = f'https://www.jobindex.dk/jobsoegning?lang=da?page={i}.html'
    jobindex_links.append(url)   
#print(jobindex_links)
html = []
job_links = []
for url in jobindex_links:
    response = requests.get(url)  
    html = response.text  
    soup = BeautifulSoup(html,'lxml')
    #one job result is given by class=jobsearch-result
    joblistings = soup.find_all('div',{'class':'jobsearch-result'})
    #print(joblistings)
    for joblisting in joblistings:
        links = re.findall("href=[\"\'](.*?)[\"\']", str(joblisting))
        link = links[1]
        job_links.append(link)
len(job_links)

40

In [20]:
for joblink in job_links:
    if joblink[-4:] == '.pdf':
        #Michelles kode:
    else:
        response = requests.get(link)
        html = response.text
        soup = BeautifulSoup(html, 'lxml')
        description = soup.get_text()
        job_descriptions.append(description)
    

https://www.stata.com/manuals13/pputexcel.pdf
